In [1]:
import os
import numpy as np
import open3d as o3d
import pyvista as pv


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:
sourceMeshPath = "results/partial_smile_arch.stl"
targetMeshPath = "assets/library/smile_arch_origin.stl"

In [10]:
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

source.compute_vertex_normals()
target.compute_vertex_normals()

TriangleMesh with 37856 points and 12620 triangles.

In [4]:
def o3d_to_pyvista(mesh):
    vertices = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.triangles)
    
    # PyVista expects faces with leading length indicators
    faces_with_count = np.column_stack((np.full(len(faces), 3), faces))
    faces_with_count = faces_with_count.flatten()
    
    # Create PyVista PolyData
    pv_mesh = pv.PolyData(vertices, faces_with_count)
    return pv_mesh

In [5]:
source_pv = o3d_to_pyvista(source)
target_pv = o3d_to_pyvista(target)

# 5. PyVista로 시각화
plotter = pv.Plotter()

# Add source mesh in blue
plotter.add_mesh(source_pv, color='lightblue', opacity=0.7, label='Source Mesh')

# Add target mesh in green
plotter.add_mesh(target_pv, color='lightgreen', opacity=0.7, label='Target Mesh')

# Add legend
plotter.add_legend()

# Show the plot
plotter.show()

Widget(value='<iframe src="http://localhost:59278/index.html?ui=P_0x207df6b7fa0_0&reconnect=auto" class="pyvis…

In [11]:
def o3d_pcd_to_pyvista(pcd):
    points = np.asarray(pcd.points)
    pv_mesh = pv.PolyData(points)  # points만 있는 PolyData 생성
    return pv_mesh

In [12]:
import open3d as o3d
import pyvista as pv
import numpy as np

def o3d_pcd_to_pyvista(pcd):
    points = np.asarray(pcd.points)
    print(f"Converting points to PyVista. Points shape: {points.shape}")
    pv_mesh = pv.PolyData(points)
    return pv_mesh

pcd_source = source.sample_points_uniformly(number_of_points=20000)
pcd_target = target.sample_points_uniformly(number_of_points=20000)

# 6. 시각화
print("\n6. Visualizing...")
plotter = pv.Plotter()
plotter.background_color = 'black'


# 소스 포인트 클라우드
source_pv = o3d_pcd_to_pyvista(pcd_source)
plotter.add_mesh(source_pv, 
                color='yellow',
                opacity=1.0,
                label='Source Points',
                render_points_as_spheres=True,
                point_size=5)  # 점 크기 줄임

# 타겟 포인트 클라우드
target_pv = o3d_pcd_to_pyvista(pcd_target)
plotter.add_mesh(target_pv, 
                color='cyan',
                opacity=1.0,
                label='Target Points',
                render_points_as_spheres=True,
                point_size=5)  # 점 크기 줄임

# 카메라 위치 설정
plotter.camera_position = 'xy'  # xy 평면에서 보기
plotter.camera.zoom(0.5)  # 확대

# 축 추가
plotter.add_axes()  # 좌표축 표시
plotter.add_bounding_box()  # 경계 상자 추가

print("\nAdding meshes to plotter completed.")
plotter.add_legend()
print("Showing plot...")
plotter.show()


6. Visualizing...
Converting points to PyVista. Points shape: (20000, 3)
Converting points to PyVista. Points shape: (20000, 3)

Adding meshes to plotter completed.
Showing plot...


Widget(value='<iframe src="http://localhost:52233/index.html?ui=P_0x11432fbe3e0_1&reconnect=auto" class="pyvis…

In [13]:
o3d.visualization.draw_geometries([pcd_source, pcd_target])


In [14]:
import open3d as o3d
import numpy as np

# ... (이전 포인트 클라우드 처리 코드는 동일) ...

# Open3D 시각화
print("\n6. Visualizing with Open3D...")

# 포인트 클라우드 색상 설정
pcd_source.paint_uniform_color([1, 0.706, 0])  # 노란색
pcd_target.paint_uniform_color([0, 0.651, 0.929])  # 하늘색

# 시각화
vis = o3d.visualization.Visualizer()
vis.create_window()

# 포인트 클라우드 추가
vis.add_geometry(pcd_source)
vis.add_geometry(pcd_target)

# 렌더링 옵션 설정
opt = vis.get_render_option()
opt.background_color = np.asarray([0, 0, 0])  # 검은 배경
opt.point_size = 3.0  # 점 크기 설정

# 초기 카메라 뷰포인트 설정
ctr = vis.get_view_control()
ctr.set_zoom(0.5)

# 시각화 실행
vis.run()
vis.destroy_window()


6. Visualizing with Open3D...


In [15]:
import open3d as o3d
import numpy as np
import copy

# 1. 메쉬 로드
print("1. Loading meshes...")
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

# 각 단계별로 확인
print(f"Source mesh: {source}")
print(f"Target mesh: {target}")

# 2. 메쉬를 포인트 클라우드로 변환
print("\n2. Converting to point clouds...")
source_pcd = source.sample_points_uniformly(number_of_points=5000)
target_pcd = target.sample_points_uniformly(number_of_points=5000)

print(f"Source point cloud: {source_pcd}")
print(f"Target point cloud: {target_pcd}")

# 먼저 각각의 포인트 클라우드를 따로 시각화해보기
print("\nViewing source point cloud alone...")
source_pcd.paint_uniform_color([1, 0, 0])  # 빨간색
o3d.visualization.draw_geometries([source_pcd])

print("\nViewing target point cloud alone...")
target_pcd.paint_uniform_color([0, 1, 0])  # 초록색
o3d.visualization.draw_geometries([target_pcd])

# 포인트 클라우드의 바운딩 박스 확인
print("\nPoint cloud bounds:")
print("Source bounds:", source_pcd.get_axis_aligned_bounding_box())
print("Target bounds:", target_pcd.get_axis_aligned_bounding_box())

# 두 포인트 클라우드 함께 시각화
print("\nViewing both point clouds together...")
o3d.visualization.draw_geometries([source_pcd, target_pcd])

1. Loading meshes...
Source mesh: TriangleMesh with 37856 points and 12620 triangles.
Target mesh: TriangleMesh with 40103 points and 13369 triangles.

2. Converting to point clouds...
Source point cloud: PointCloud with 5000 points.
Target point cloud: PointCloud with 5000 points.

Viewing source point cloud alone...

Viewing target point cloud alone...

Point cloud bounds:
Source bounds: AxisAlignedBoundingBox: min: (-22.7629, -14.2567, 0.130808), max: (22.2597, 0.118814, 13.4192)
Target bounds: AxisAlignedBoundingBox: min: (-20.0642, -9.60143, 0.0613093), max: (19.4479, 0.620935, 17.7709)

Viewing both point clouds together...


In [16]:
import open3d as o3d
import numpy as np
import copy

# 1. 메쉬 로드 및 포인트 클라우드 변환
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

# 더 많은 포인트로 샘플링
source_pcd = source.sample_points_uniformly(number_of_points=20000)
target_pcd = target.sample_points_uniformly(number_of_points=20000)

# 2. Y=0 평면을 기준으로 정렬 (교합면 정렬)
def align_to_occlusal_plane(pcd):
    points = np.asarray(pcd.points)
    y_max = np.max(points[:, 1])
    pcd.translate([0, -y_max, 0])  # Y 최대값을 0으로 이동
    return pcd

source_pcd = align_to_occlusal_plane(source_pcd)
target_pcd = align_to_occlusal_plane(target_pcd)

# 3. Voxel 다운샘플링
voxel_size = 0.3  # 메쉬 크기를 고려한 voxel 크기
source_down = source_pcd.voxel_down_sample(voxel_size=voxel_size)
target_down = target_pcd.voxel_down_sample(voxel_size=voxel_size)

# 4. 법선 벡터 계산
source_down.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
target_down.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))

# 5. ICP 정합 시도 (여러 초기 위치에서)
def try_icp_registration(source, target, init_transform):
    threshold = voxel_size
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, init_transform,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
    return reg_p2p

# Y축 중심 회전 시도
best_result = None
best_fitness = 0

for y_angle in np.linspace(0, 2*np.pi, 8):  # 8개의 다른 각도에서 시도
    # Y축 회전 행렬
    init_transform = np.array([
        [np.cos(y_angle), 0, np.sin(y_angle), 0],
        [0, 1, 0, 0],
        [-np.sin(y_angle), 0, np.cos(y_angle), 0],
        [0, 0, 0, 1]
    ])
    
    result = try_icp_registration(source_down, target_down, init_transform)
    print(f"Angle: {y_angle:.2f}, Fitness: {result.fitness:.4f}")
    
    if result.fitness > best_fitness:
        best_fitness = result.fitness
        best_result = result

print("\nBest registration result:")
print(best_result)

# 6. 결과 시각화
source_pcd.paint_uniform_color([1, 0, 0])      # 빨간색
target_pcd.paint_uniform_color([0, 1, 0])      # 초록색

# 변환 적용
source_transformed = copy.deepcopy(source_pcd)
source_transformed.transform(best_result.transformation)
source_transformed.paint_uniform_color([0, 0, 1])  # 파란색

print("\nViewing registration result...")
o3d.visualization.draw_geometries([source_transformed, target_pcd])

Angle: 0.00, Fitness: 0.2753
Angle: 0.90, Fitness: 0.1910
Angle: 1.80, Fitness: 0.1588
Angle: 2.69, Fitness: 0.0538
Angle: 3.59, Fitness: 0.0389
Angle: 4.49, Fitness: 0.1609
Angle: 5.39, Fitness: 0.1467
Angle: 6.28, Fitness: 0.2753

Best registration result:
RegistrationResult with fitness=2.752541e-01, inlier_rmse=1.961132e-01, and correspondence_set size of 2248
Access transformation to get result.

Viewing registration result...


In [17]:
import open3d as o3d
import numpy as np
import copy

# 1. 메쉬 로드 및 포인트 클라우드 변환
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

source_pcd = source.sample_points_uniformly(number_of_points=20000)
target_pcd = target.sample_points_uniformly(number_of_points=20000)

# 2. Y=0 평면 정렬
def align_to_occlusal_plane(pcd):
    points = np.asarray(pcd.points)
    y_max = np.max(points[:, 1])
    pcd.translate([0, -y_max, 0])
    return pcd

source_pcd = align_to_occlusal_plane(source_pcd)
target_pcd = align_to_occlusal_plane(target_pcd)

# 3. Voxel 다운샘플링
voxel_size = 0.3
source_down = source_pcd.voxel_down_sample(voxel_size=voxel_size)
target_down = target_pcd.voxel_down_sample(voxel_size=voxel_size)

# 4. 법선 벡터 계산
source_down.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
target_down.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))

# 5. ICP 정합 시도 (여러 위치에서)
def try_icp_registration(source, target, init_transform):
    threshold = voxel_size
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, init_transform,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000))
    return reg_p2p

# X축 이동 범위 설정 (-5mm에서 +5mm까지)
x_translations = np.linspace(-5, 5, 5)  # -5mm에서 5mm까지 5개 지점
x_rotations = np.linspace(-np.pi/8, np.pi/8, 5)  # -45도에서 45도까지 5개 지점
y_rotations = np.linspace(-np.pi/8, np.pi/8, 5)  # -45도에서 45도까지 5개 지점
z_rotations = np.linspace(-np.pi/8, np.pi/8, 5)  # -45도에서 45도까지 5개 지점
best_result = None
best_fitness = 0

print("Trying different positions and rotations...")
for x_trans in x_translations:
    for x_angle in x_rotations:
        for y_angle in y_rotations:
            for z_angle in z_rotations:
                # X, Y, Z축 회전 + X축 이동 변환 행렬
                Rx = np.array([
                    [1, 0, 0],
                    [0, np.cos(x_angle), -np.sin(x_angle)],
                    [0, np.sin(x_angle), np.cos(x_angle)]
                ])
                Ry = np.array([
                    [np.cos(y_angle), 0, np.sin(y_angle)],
                    [0, 1, 0],
                    [-np.sin(y_angle), 0, np.cos(y_angle)]
                ])
                Rz = np.array([
                    [np.cos(z_angle), -np.sin(z_angle), 0],
                    [np.sin(z_angle), np.cos(z_angle), 0],
                    [0, 0, 1]
                ])
                
                # 회전 행렬 결합
                R = Rz @ Ry @ Rx
                
                init_transform = np.eye(4)
                init_transform[:3, :3] = R
                init_transform[0, 3] = x_trans
                
                result = try_icp_registration(source_down, target_down, init_transform)
                #print(f"X translation: {x_trans:.1f}, Angles: x={np.degrees(x_angle):.1f}, y={np.degrees(y_angle):.1f}, z={np.degrees(z_angle):.1f}, Fitness: {result.fitness:.4f}")
                
                if result.fitness > best_fitness:
                    best_fitness = result.fitness
                    best_result = result

print("\nBest registration result:")
print(best_result)
print(f"Best fitness: {best_fitness}")

# 6. 결과 시각화
source_pcd.paint_uniform_color([1, 0, 0])      # 빨간색 (원본)
target_pcd.paint_uniform_color([0, 1, 0])      # 초록색 (타겟)

# 최적의 변환 적용
source_transformed = copy.deepcopy(source_pcd)
source_transformed.transform(best_result.transformation)
source_transformed.paint_uniform_color([0, 0, 1])  # 파란색 (변환됨)

print("\nViewing original state...")
o3d.visualization.draw_geometries([source_pcd, target_pcd])

print("\nViewing registration result...")
o3d.visualization.draw_geometries([source_transformed, target_pcd])

# 최적의 변환 행렬 출력
print("\nBest transformation matrix:")
print(np.array2string(best_result.transformation, precision=6, suppress_small=True))

# 변환 행렬 분석
rotation = best_result.transformation[:3, :3]
translation = best_result.transformation[:3, 3]

# 회전 각도 계산 (Y축 회전)
angle_y = np.arctan2(rotation[0, 2], rotation[0, 0])
angle_degrees = np.degrees(angle_y)

print("\nTransformation analysis:")
print(f"Translation vector [x, y, z]: {translation}")
print(f"Rotation angle around Y-axis: {angle_degrees:.2f} degrees")

# 정합 품질 메트릭
print("\nRegistration quality metrics:")
print(f"Fitness: {best_result.fitness}")
print(f"Inlier RMSE: {best_result.inlier_rmse}")

Trying different positions and rotations...

Best registration result:
RegistrationResult with fitness=4.458932e-01, inlier_rmse=1.963200e-01, and correspondence_set size of 3659
Access transformation to get result.
Best fitness: 0.4458932488423105

Viewing original state...

Viewing registration result...

Best transformation matrix:
[[ 0.99689   0.074091 -0.026845  3.727928]
 [-0.07202   0.994854  0.071265 -0.228034]
 [ 0.031987 -0.06911   0.997096  0.10538 ]
 [ 0.        0.        0.        1.      ]]

Transformation analysis:
Translation vector [x, y, z]: [ 3.72792812 -0.22803386  0.10538001]
Rotation angle around Y-axis: -1.54 degrees

Registration quality metrics:
Fitness: 0.4458932488423105
Inlier RMSE: 0.19631996845493607


In [18]:
import open3d as o3d
import numpy as np
import copy

# 주축 정렬까지는 동일
print("1. Initial principal axis alignment...")
source_mean, source_axes = get_principal_axes(source_pcd)
target_mean, target_axes = get_principal_axes(target_pcd)

R = target_axes @ source_axes.T
t = target_mean - R @ source_mean

init_transform = np.eye(4)
init_transform[:3, :3] = R
init_transform[:3, 3] = t

# 미세 조정을 위한 추가 변환 시도
print("\n2. Fine-tuning with multiple positions...")
x_translations = np.linspace(-2, 2, 5)  # ±2mm
x_rotations = np.linspace(-np.pi/6, np.pi/6, 5)  # ±30도
y_rotations = np.linspace(-np.pi/6, np.pi/6, 5)  # ±30도
z_rotations = np.linspace(-np.pi/6, np.pi/6, 5)  # ±30도

best_result = None
best_fitness = 0

for x_trans in x_translations:
    for x_angle in x_rotations:
        for y_angle in y_rotations:
            for z_angle in z_rotations:
                # X축 회전 행렬
                Rx = np.array([
                    [1, 0, 0],
                    [0, np.cos(x_angle), -np.sin(x_angle)],
                    [0, np.sin(x_angle), np.cos(x_angle)]
                ])
                
                # Y축 회전 행렬
                Ry = np.array([
                    [np.cos(y_angle), 0, np.sin(y_angle)],
                    [0, 1, 0],
                    [-np.sin(y_angle), 0, np.cos(y_angle)]
                ])
                
                # Z축 회전 행렬
                Rz = np.array([
                    [np.cos(z_angle), -np.sin(z_angle), 0],
                    [np.sin(z_angle), np.cos(z_angle), 0],
                    [0, 0, 1]
                ])
                
                # 전체 회전 행렬 (Z -> Y -> X 순서로 적용)
                R_fine = Rx @ Ry @ Rz
                
                # 미세 조정 변환 행렬
                fine_transform = np.eye(4)
                fine_transform[:3, :3] = R_fine
                fine_transform[0, 3] = x_trans
                
                # 초기 정렬에 미세 조정을 결합
                combined_transform = fine_transform @ init_transform
                
                result = o3d.pipelines.registration.registration_icp(
                    source_pcd, target_pcd,
                    0.5,  # threshold
                    combined_transform,
                    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100)
                )
                
                print(f"X_trans: {x_trans:.1f}, X_rot: {np.degrees(x_angle):.1f}°, Y_rot: {np.degrees(y_angle):.1f}°, Z_rot: {np.degrees(z_angle):.1f}°, Fitness: {result.fitness:.4f}")
                
                if result.fitness > best_fitness:
                    best_fitness = result.fitness
                    best_result = result

print("\nBest registration result:")
print(best_result)
print(f"\nBest fitness: {best_fitness}")

# 결과 시각화
source_pcd.paint_uniform_color([1, 0, 0])      # 빨간색
target_pcd.paint_uniform_color([0, 1, 0])      # 초록색

source_transformed = copy.deepcopy(source_pcd)
source_transformed.transform(best_result.transformation)
source_transformed.paint_uniform_color([0, 0, 1])  # 파란색

# 좌표축 추가
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0)

print("\nViewing final result...")
o3d.visualization.draw_geometries([source_transformed, target_pcd, coordinate_frame])

1. Initial principal axis alignment...


NameError: name 'get_principal_axes' is not defined

In [19]:
import open3d as o3d
import numpy as np
import copy

def draw_registration_result(source, target, transformation=None):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0, 0])
    target_temp.paint_uniform_color([0, 1, 0])
    
    if transformation is not None:
        source_temp.transform(transformation)
    
    coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0)
    
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(source_temp)
    vis.add_geometry(target_temp)
    vis.add_geometry(coord)
    
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    opt.point_size = 2.0
    
    vis.run()
    vis.destroy_window()

def prepare_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
    return pcd_down

def fast_registration(source, target):
    source_points = np.asarray(source.points)
    target_points = np.asarray(target.points)
    
    source_mean = np.mean(source_points, axis=0)
    target_mean = np.mean(target_points, axis=0)
    
    init_translation = target_mean - source_mean
    
    # 각 축별 회전 각도 (-15도에서 +15도, 7.5도 간격)
    angles = np.linspace(-np.pi/12, np.pi/12, 5)  # [-15°, -7.5°, 0°, 7.5°, 15°]
    
    # X축 이동 범위
    x_offsets = list(range(-10, 11, 5))
    
    best_fitness = 0
    best_transform = None
    total_tries = len(angles) * len(angles) * len(angles) * len(x_offsets)
    current_try = 0
    
    estimation = o3d.pipelines.registration.TransformationEstimationPointToPlane()
    criteria = o3d.pipelines.registration.ICPConvergenceCriteria(
        relative_fitness=1e-6,
        relative_rmse=1e-6,
        max_iteration=30
    )
    
    # 각 축별 회전 행렬 생성 함수
    def create_rotation_matrix(angle_x, angle_y, angle_z):
        # X축 회전
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(angle_x), -np.sin(angle_x)],
            [0, np.sin(angle_x), np.cos(angle_x)]
        ])
        
        # Y축 회전
        Ry = np.array([
            [np.cos(angle_y), 0, np.sin(angle_y)],
            [0, 1, 0],
            [-np.sin(angle_y), 0, np.cos(angle_y)]
        ])
        
        # Z축 회전
        Rz = np.array([
            [np.cos(angle_z), -np.sin(angle_z), 0],
            [np.sin(angle_z), np.cos(angle_z), 0],
            [0, 0, 1]
        ])
        
        # 회전 순서: X -> Y -> Z
        return Rz @ Ry @ Rx
    
    for angle_x in angles:  # X축 회전 (빨간색 축)
        for angle_y in angles:  # Y축 회전 (초록색 축)
            for angle_z in angles:  # Z축 회전 (파란색 축)
                for x_offset in x_offsets:
                    current_try += 1
                    print(f"\rTrying combination {current_try}/{total_tries} "
                          f"(X: {np.degrees(angle_x):.1f}°, "
                          f"Y: {np.degrees(angle_y):.1f}°, "
                          f"Z: {np.degrees(angle_z):.1f}°, "
                          f"offset: {x_offset}mm)", end="")
                    
                    # 회전 행렬 생성
                    R = create_rotation_matrix(angle_x, angle_y, angle_z)
                    
                    # 변환 행렬 생성
                    transform = np.eye(4)
                    transform[:3, :3] = R
                    transform[:3, 3] = init_translation + np.array([x_offset, 0, 0])
                    
                    # Trimmed ICP 실행
                    result = o3d.pipelines.registration.registration_icp(
                        source, target,
                        max_correspondence_distance=1.0,
                        init=transform,
                        estimation_method=estimation,
                        criteria=criteria
                    )
                    
                    if result.fitness > best_fitness:
                        best_fitness = result.fitness
                        best_transform = result.transformation
                        print(f"\nNew best fitness: {best_fitness:.6f} "
                              f"(X: {np.degrees(angle_x):.1f}°, "
                              f"Y: {np.degrees(angle_y):.1f}°, "
                              f"Z: {np.degrees(angle_z):.1f}°, "
                              f"offset: {x_offset}mm)")
    
    print("\n")
    return best_transform, best_fitness

# 메인 코드는 이전과 동일
print("1. Loading point clouds...")
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

voxel_size = 0.5
source_pcd = source.sample_points_uniformly(number_of_points=50000)
target_pcd = target.sample_points_uniformly(number_of_points=50000)

print("2. Preparing point clouds...")
source_down = prepare_point_cloud(source_pcd, voxel_size)
target_down = prepare_point_cloud(target_pcd, voxel_size)

print("3. Running fast registration...")
best_transform, initial_fitness = fast_registration(source_down, target_down)

print(f"Initial alignment complete.")
print(f"Initial fitness: {initial_fitness}")

print("\n4. Final refinement...")
final_result = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd,
    max_correspondence_distance=voxel_size * 0.4,
    init=best_transform,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    criteria=o3d.pipelines.registration.ICPConvergenceCriteria(
        relative_fitness=1e-6,
        relative_rmse=1e-6,
        max_iteration=50
    )
)

print(f"Final fitness: {final_result.fitness}")
print(f"Final RMSE: {final_result.inlier_rmse}")

print("\nViewing final result...")
draw_registration_result(source_pcd, target_pcd, final_result.transformation)

print("\nFinal transformation matrix:")
print(np.array2string(final_result.transformation, precision=6, suppress_small=True))

1. Loading point clouds...
2. Preparing point clouds...
3. Running fast registration...
Trying combination 1/625 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: -10mm)
New best fitness: 0.251975 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: -10mm)
Trying combination 2/625 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: -5mm)
New best fitness: 0.489682 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: -5mm)
Trying combination 3/625 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: 0mm)
New best fitness: 0.716178 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: 0mm)
Trying combination 9/625 (X: -15.0°, Y: -15.0°, Z: -7.5°, offset: 5mm)))
New best fitness: 0.739363 (X: -15.0°, Y: -15.0°, Z: -7.5°, offset: 5mm)
Trying combination 23/625 (X: -15.0°, Y: -15.0°, Z: 15.0°, offset: 0mm)))
New best fitness: 0.748280 (X: -15.0°, Y: -15.0°, Z: 15.0°, offset: 0mm)
Trying combination 32/625 (X: -15.0°, Y: -7.5°, Z: -7.5°, offset: -5mm)))
New best fitness: 0.751338 (X: -15.0°, Y: -7.5°, Z: -7.5°, offset: -5mm)
Trying combination 59/625 (X

In [35]:
import open3d as o3d
import numpy as np
import copy
import os

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def save_registration_view(source, target, transformation=None, filename="view.png", title="Registration View"):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0, 0])    # 빨간색 (소스)
    target_temp.paint_uniform_color([0, 1, 0])    # 초록색 (타겟)
    
    if transformation is not None:
        source_temp.transform(transformation)
    
    # 좌표축 추가
    coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0)
    
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=1024, height=768)  # 해상도 증가
    
    # 지오메트리 추가
    vis.add_geometry(source_temp)
    vis.add_geometry(target_temp)
    vis.add_geometry(coord)
    
    # 렌더링 옵션 설정
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])  # 검은 배경
    opt.point_size = 2.0
    
    # 쿼터뷰 카메라 설정
    ctr = vis.get_view_control()
    
    # 카메라 위치 설정 (쿼터뷰)
    # front: 카메라가 바라보는 방향 벡터
    # up: 카메라의 상단 방향 벡터
    # zoom: 확대/축소 레벨
    front = [-0.2, 1, 1]  # 정면과 위에서 약간 비스듬히 보는 각도
    up = [0, -0.5, 1]           # 약간 기울어진 상단 방향
    
    ctr.set_front(front)
    ctr.set_up(up)
    ctr.set_lookat([-5, -5, 5])  # 중심점 바라보기
    ctr.set_zoom(0.7)          # 줌 레벨 조정
    
    # 이미지 캡처 및 저장
    vis.poll_events()
    vis.update_renderer()
    vis.capture_screen_image(filename, do_render=True)
    vis.destroy_window()


# 메인 프로세스
print("1. Loading point clouds...")
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

# 결과 저장 디렉토리 생성
ensure_dir("registration_results")

# 1. 원본 메쉬 시각화
source_pcd = source.sample_points_uniformly(number_of_points=20000)
target_pcd = target.sample_points_uniformly(number_of_points=20000)
save_registration_view(source_pcd, target_pcd, 
                      filename="registration_results/1_initial_state.png",
                      title="Initial State")

# 2. 포인트 클라우드 준비
voxel_size = 0.5
source_down = source_pcd.voxel_down_sample(voxel_size)
target_down = target_pcd.voxel_down_sample(voxel_size)

# 법선 벡터 계산
source_down.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
target_down.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))

save_registration_view(source_down, target_down,
                      filename="registration_results/2_downsampled.png",
                      title="Downsampled Point Clouds")

# 3. 초기 정렬 시도 (여러 방향)
angles = np.linspace(-np.pi/12, np.pi/12, 10)  # [-15°, -7.5°, 0°, 7.5°, 15°]
x_offsets = [-10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10]

best_fitness = 0
best_transform = None
current_try = 0
total_tries = len(angles) * len(angles) * len(angles) * len(x_offsets)

# 진행 상황을 5단계로 나누어 저장
progress_steps = total_tries // 5

for i, angle_x in enumerate(angles):
    for angle_y in angles:
        for angle_z in angles:
            for x_offset in x_offsets:
                current_try += 1
                
                # 회전 행렬 생성
                Rx = np.array([
                    [1, 0, 0],
                    [0, np.cos(angle_x), -np.sin(angle_x)],
                    [0, np.sin(angle_x), np.cos(angle_x)]
                ])
                
                Ry = np.array([
                    [np.cos(angle_y), 0, np.sin(angle_y)],
                    [0, 1, 0],
                    [-np.sin(angle_y), 0, np.cos(angle_y)]
                ])
                
                Rz = np.array([
                    [np.cos(angle_z), -np.sin(angle_z), 0],
                    [np.sin(angle_z), np.cos(angle_z), 0],
                    [0, 0, 1]
                ])
                
                R = Rz @ Ry @ Rx
                transform = np.eye(4)
                transform[:3, :3] = R
                transform[0, 3] = x_offset
                
                result = o3d.pipelines.registration.registration_icp(
                    source_down, target_down,
                    max_correspondence_distance=1.0,
                    init=transform,
                    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
                    criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=30)
                )
                
                if result.fitness > best_fitness:
                    best_fitness = result.fitness
                    best_transform = result.transformation
                    
                    # 새로운 최적 결과를 찾을 때마다 저장
                    save_registration_view(source_pcd, target_pcd, best_transform,
                                        f"registration_results/3_progress_{current_try}.png",
                                        f"Progress {current_try}/{total_tries}")

# 4. 최종 미세 조정
final_result = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd,
    max_correspondence_distance=voxel_size * 0.4,
    init=best_transform,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    criteria=o3d.pipelines.registration.ICPConvergenceCriteria(
        relative_fitness=1e-6,
        relative_rmse=1e-6,
        max_iteration=50
    )
)

# 5. 최종 결과 저장
save_registration_view(source_pcd, target_pcd, final_result.transformation,
                      "registration_results/4_final_result.png",
                      "Final Registration Result")

print("\nVisualization results have been saved to 'registration_results' directory")
print("Files saved:")
print("1. 1_initial_state.png - Original point clouds")
print("2. 2_downsampled.png - Downsampled point clouds")
print("3. 3_progress_*.png - Registration progress")
print("4. 4_final_result.png - Final registration result")

# 최종 변환 행렬 저장
with open("registration_results/transformation_matrix.txt", "w") as f:
    f.write("Final transformation matrix:\n")
    f.write(np.array2string(final_result.transformation, precision=6, suppress_small=True))
    f.write("\n\nRegistration metrics:\n")
    f.write(f"Fitness: {final_result.fitness}\n")
    f.write(f"Inlier RMSE: {final_result.inlier_rmse}\n")

1. Loading point clouds...

Visualization results have been saved to 'registration_results' directory
Files saved:
1. 1_initial_state.png - Original point clouds
2. 2_downsampled.png - Downsampled point clouds
3. 3_progress_*.png - Registration progress
4. 4_final_result.png - Final registration result


In [9]:
import open3d as o3d
import numpy as np
import copy
import time

def draw_registration_result_live(vis, source, target, transformation=None, fitness=None, status="Processing"):
    """실시간 시각화를 위한 업데이트 함수"""
    source_temp = copy.deepcopy(source)
    if transformation is not None:
        source_temp.transform(transformation)
    
    # 한 번만 clear_geometries 호출
    vis.clear_geometries()
    
    # 좌표축 추가
    coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0)
    
    # 색상 설정
    source_temp.paint_uniform_color([1, 0, 0])  # 빨간색 (소스)
    target.paint_uniform_color([0, 1, 0])      # 초록색 (타겟)
    
    # 지오메트리 추가
    vis.add_geometry(source_temp)
    vis.add_geometry(target)
    vis.add_geometry(coord)
    
    # 렌더링 옵션 설정
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    opt.point_size = 2.0
    
    # 카메라 설정
    ctr = vis.get_view_control()
    front = [-0.0, 4, 4]  # X축 음의 방향에서 약간 아래쪽으로
    up = [0, 0, 1]    
    ctr.set_front(front)
    ctr.set_up(up)
    ctr.set_lookat([0, 0, 4])
    ctr.set_zoom(0.5)
    
    # 상태 정보 출력 (콘솔에)
    if fitness is not None:
        print(f"{status}\nFitness: {fitness:.6f}")
    else:
        print(status)
    
    return False


def live_registration_process(source, target, voxel_size=0.5):
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Registration Process", width=1920, height=1080)
    
    # 1단계: 초기 정렬
    print("\nStage 1: Initial Alignment...")
    angles = np.linspace(-np.pi/20, np.pi/20, 5)
    x_offsets = np.arange(-2, 2.1, 0.5)
    best_fitness = 0
    best_transform = np.eye(4)
    best_result = None  # best_result 초기화 추가
    current_transform = np.eye(4)  # current_transform 초기화 추가
    
    # 초기 상태 표시
    draw_registration_result_live(vis, source, target, None, None, "Initial Alignment - Searching Best Position")
    vis.poll_events()
    vis.update_renderer()


    for angle_x in angles:
        for angle_y in angles:
            for angle_z in angles:
                for x_offset in x_offsets:
                    Rx = np.array([
                        [1, 0, 0],
                        [0, np.cos(angle_x), -np.sin(angle_x)],
                        [0, np.sin(angle_x), np.cos(angle_x)]
                    ])
                    Ry = np.array([
                        [np.cos(angle_y), 0, np.sin(angle_y)],
                        [0, 1, 0],
                        [-np.sin(angle_y), 0, np.cos(angle_y)]
                    ])
                    Rz = np.array([
                        [np.cos(angle_z), -np.sin(angle_z), 0],
                        [np.sin(angle_z), np.cos(angle_z), 0],
                        [0, 0, 1]
                    ])
                    
                    R = Rz @ Ry @ Rx
                    transform = np.eye(4)
                    transform[:3, :3] = R
                    transform[0, 3] = x_offset
                    
                    result = o3d.pipelines.registration.registration_icp(
                        source, target, voxel_size, transform,
                        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1))
                    
                    # 모든 시도를 시각화 (best가 아니어도)
                    status = f"Current Angles: ({np.degrees(angle_x):.1f}°, {np.degrees(angle_y):.1f}°, {np.degrees(angle_z):.1f}°)\n" \
                            f"Offset: {x_offset:.1f}mm\n" \
                            f"Current Fitness: {result.fitness:.6f}\n" \
                            f"Best Fitness: {best_fitness:.6f}"
                    
                    print(status)
                    draw_registration_result_live(vis, source, target, transform, best_fitness, status)
                    vis.poll_events()
                    vis.update_renderer()
                    time.sleep(0.1)

                    if result.fitness > best_fitness:
                        best_fitness = result.fitness
                        best_transform = result.transformation
                        best_result = result  # best_result 업데이트 추가
                        status = f"Initial Alignment\nBest Position Found\nAngles: ({np.degrees(angle_x):.1f}°, {np.degrees(angle_y):.1f}°, {np.degrees(angle_z):.1f}°)\nOffset: {x_offset}mm"
                    
    
    # 2단계: 점진적 ICP 정합
    print("\nStage 2: Progressive ICP Registration...")
    for iteration in range(50):
        result = o3d.pipelines.registration.registration_icp(
            source, target,
            voxel_size * 0.4,
            current_transform,
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            o3d.pipelines.registration.ICPConvergenceCriteria(
                relative_fitness=1e-6,
                relative_rmse=1e-6,
                max_iteration=1
            )
        )
        
        if result.fitness > best_fitness:
            best_fitness = result.fitness
            best_result = result  # best_result 업데이트
            current_transform = result.transformation
            status = f"Fine Registration (ICP)\nIteration: {iteration}\nBest Fitness: {best_fitness:.6f}"
        draw_registration_result_live(vis, source, target, current_transform, best_fitness, status)
        vis.poll_events()
        vis.update_renderer()
        # time.sleep(0.01)
        
        if iteration > 0 and abs(result.fitness - best_fitness) < 1e-6:
            break
    
    # 3단계: 최종 미세 조정
    print("\nStage 3: Final Refinement...")
    current_transform = best_result.transformation
    
    for iteration in range(100):  # 100번 반복을 나누어서 수행
        result = o3d.pipelines.registration.registration_icp(
            source, target,
            voxel_size * 0.2,
            current_transform,
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            o3d.pipelines.registration.ICPConvergenceCriteria(
                relative_fitness=1e-8,
                relative_rmse=1e-8,
                max_iteration=1  # 한 번씩 실행
            )
        )
        
        current_transform = result.transformation
        
        # 진행 상황 시각화
        status = f"Final Refinement\nIteration: {iteration}/100\nFitness: {result.fitness:.6f}\nRMSE: {result.inlier_rmse:.6f}"
        draw_registration_result_live(vis, source, target, current_transform, result.fitness, status)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.01)  # 애니메이션 효과를 위한 지연
        
        # 수렴 조건 확인
        if iteration > 0 and abs(result.fitness - best_fitness) < 1e-8:
            break
        
        # 더 좋은 결과면 저장
        if result.fitness > best_fitness:
            best_fitness = result.fitness
            best_result = result
            
    # # 3단계: 최종 미세 조정
    # print("\nStage 3: Final Refinement...")
    # final_result = o3d.pipelines.registration.registration_icp(
    #     source, target,
    #     voxel_size * 0.2,
    #     best_result.transformation,  # best_result는 이제 항상 존재
    #     o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    #     o3d.pipelines.registration.ICPConvergenceCriteria(
    #         relative_fitness=1e-8,
    #         relative_rmse=1e-8,
    #         max_iteration=100
    #     )
    # )
    
    # # 최종 결과가 이전 best보다 더 좋은 경우에만 적용
    # if final_result.fitness > best_fitness:
    #     best_fitness = final_result.fitness
    #     best_result = final_result
    #     final_status = f"Registration Complete!\nFinal Fitness: {best_fitness:.6f}\nRMSE: {best_result.inlier_rmse:.6f}\n\nPress 'q' to exit..."
    # else:
    #     final_status = f"Registration Complete!\nBest Fitness: {best_fitness:.6f}\nRMSE: {best_result.inlier_rmse:.6f}\n(Final refinement did not improve result)\n\nPress 'q' to exit..."
    
    # 최종 결과 표시
    draw_registration_result_live(vis, source, target, best_result.transformation, best_fitness)
    vis.poll_events()
    vis.update_renderer()
    
    print("\nRegistration completed. Press 'q' to exit...")
    
    while True:
        if not vis.poll_events():
            break
        vis.update_renderer()
        time.sleep(0.1)
    
    vis.destroy_window()
    return best_result.transformation

# 메인 실행 코드
print("1. Loading point clouds...")
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

# 포인트 클라우드 생성 및 전처리
source_pcd = source.sample_points_uniformly(number_of_points=20000)
target_pcd = target.sample_points_uniformly(number_of_points=20000)

# 다운샘플링
voxel_size = 0.8
source_down = source_pcd.voxel_down_sample(voxel_size=voxel_size)
target_down = target_pcd.voxel_down_sample(voxel_size=voxel_size)

# 법선 벡터 계산
source_down.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
target_down.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))

print("\n2. Starting full registration process with visualization...")
final_transformation = live_registration_process(source_down, target_down, voxel_size)

print("\nFinal transformation matrix:")
print(np.array2string(final_transformation, precision=6, suppress_small=True))

1. Loading point clouds...

2. Starting full registration process with visualization...

Stage 1: Initial Alignment...
Initial Alignment - Searching Best Position
Current Angles: (-9.0°, -9.0°, -9.0°)
Offset: -2.0mm
Current Fitness: 0.136129
Best Fitness: 0.000000
Current Angles: (-9.0°, -9.0°, -9.0°)
Offset: -2.0mm
Current Fitness: 0.136129
Best Fitness: 0.000000
Fitness: 0.000000
Current Angles: (-9.0°, -9.0°, -9.0°)
Offset: -1.5mm
Current Fitness: 0.140000
Best Fitness: 0.136129
Current Angles: (-9.0°, -9.0°, -9.0°)
Offset: -1.5mm
Current Fitness: 0.140000
Best Fitness: 0.136129
Fitness: 0.136129
Current Angles: (-9.0°, -9.0°, -9.0°)
Offset: -1.0mm
Current Fitness: 0.136129
Best Fitness: 0.140000
Current Angles: (-9.0°, -9.0°, -9.0°)
Offset: -1.0mm
Current Fitness: 0.136129
Best Fitness: 0.140000
Fitness: 0.140000
Current Angles: (-9.0°, -9.0°, -9.0°)
Offset: -0.5mm
Current Fitness: 0.147097
Best Fitness: 0.140000
Current Angles: (-9.0°, -9.0°, -9.0°)
Offset: -0.5mm
Current Fitness:

# 속도 개선판

In [8]:
import open3d as o3d
import numpy as np
import copy
import time

def draw_registration_result_live(vis, source, target, transformation=None, title="Registration"):
    """실시간으로 정합 과정을 시각화합니다."""
    source_temp = copy.deepcopy(source)
    # 타겟은 복사하지 않고 원본 사용
    source_temp.paint_uniform_color([1, 0, 0])  # 빨간색 (소스)
    target.paint_uniform_color([0, 1, 0])  # 초록색 (타겟)
    
    if transformation is not None:
        source_temp.transform(transformation)
    
    # 기존 지오메트리 제거 및 새로운 지오메트리 추가
    vis.clear_geometries()
    vis.add_geometry(source_temp)
    vis.add_geometry(target)
    
    # 좌표축 추가
    coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0)
    vis.add_geometry(coord)
    
    # 렌더링 옵션 설정
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])  # 검은 배경
    opt.point_size = 3.0
    
    # 카메라 설정
    ctr = vis.get_view_control()
    front = [-0.0, 4, 4]  # X축 음의 방향에서 약간 아래쪽으로
    up = [0, 0, 1]    
    ctr.set_front(front)
    ctr.set_up(up)
    ctr.set_lookat([0, 0, 4])
    ctr.set_zoom(0.5)

    # 업데이트
    vis.poll_events()
    vis.update_geometry(source_temp)
    vis.update_geometry(target)
    vis.update_geometry(coord)
    vis.update_renderer()

def align_to_occlusal_plane(pcd):
    """교합면을 기준으로 정렬합니다."""
    points = np.asarray(pcd.points)
    y_max = np.max(points[:, 1])
    pcd.translate([0, -y_max, 0])  # Y 최대값을 0으로 이동
    return pcd

def get_principal_axes(pcd):
    """포인트 클라우드의 주축과 중심점을 계산합니다."""
    points = np.asarray(pcd.points)
    mean = np.mean(points, axis=0)
    centered = points - mean
    cov = np.cov(centered.T)
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    # 주축 정렬 (큰 고유값 순서대로)
    idx = eigenvalues.argsort()[::-1]
    axes = eigenvectors[:, idx]
    return mean, axes

def fast_registration(source, target, vis=None):
    """주축 정렬을 사용한 빠른 정합을 수행하고 과정을 시각화합니다."""
    
    # 1. 교합면 정렬 (소스에만 적용)
    print("1. Aligning source to occlusal plane...")
    source = align_to_occlusal_plane(source)
    # 타겟은 원본 그대로 유지
    
    if vis is not None:
        draw_registration_result_live(vis, source, target, None, "After Occlusal Alignment")
        time.sleep(1)
    
    # 2. 초기 ICP 실행
    print("2. Running initial ICP...")
    result = o3d.pipelines.registration.registration_icp(
        source, target,
        2.0,  # 초기 threshold
        np.eye(4),  # 단위 행렬 (변환 없음)
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(
            relative_fitness=1e-6,
            relative_rmse=1e-6,
            max_iteration=1000
        )
    )
    
    if vis is not None:
        draw_registration_result_live(vis, source, target, result.transformation, "After Initial ICP")
        time.sleep(0.1)
    
    # 3. 중간 미세 조정
    print("3. Intermediate refinement...")
    intermediate_result = o3d.pipelines.registration.registration_icp(
        source, target,
        0.5,  # 중간 threshold
        result.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(
            relative_fitness=1e-7,
            relative_rmse=1e-7,
            max_iteration=1500
        )
    )
    
    if vis is not None:
        draw_registration_result_live(vis, source, target, intermediate_result.transformation, "After Intermediate Refinement")
        time.sleep(0.1)
    
    # 4. 최종 미세 조정
    print("4. Final refinement...")
    final_result = o3d.pipelines.registration.registration_icp(
        source, target,
        0.2,  # 더 작은 threshold
        intermediate_result.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(
            relative_fitness=1e-8,
            relative_rmse=1e-8,
            max_iteration=2000
        )
    )
    
    if vis is not None:
        draw_registration_result_live(vis, source, target, final_result.transformation, "Final Result")
        vis.run()
    
    return final_result.transformation, final_result.fitness

# 사용 예시:
if __name__ == "__main__":
    # 시각화 창 생성
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    sourceMeshPath = "results/partial_smile_arch.stl"   
    targetMeshPath = "assets/library/smile_arch_origin.stl"

    source = o3d.io.read_triangle_mesh(sourceMeshPath)
    target = o3d.io.read_triangle_mesh(targetMeshPath)

    # 포인트 클라우드 생성 및 전처리
    source_pcd = source.sample_points_uniformly(number_of_points=20000)
    target_pcd = target.sample_points_uniformly(number_of_points=20000)
    
    # 포인트 클라우드 준비
    voxel_size = 0.8
    source_down = source_pcd.voxel_down_sample(voxel_size=voxel_size)
    target_down = target_pcd.voxel_down_sample(voxel_size=voxel_size)
    
    # 법선 벡터 계산
    source_down.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
    target_down.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
    
    # 초기 상태 시각화
    draw_registration_result_live(vis, source_down, target_down, None, "Initial State")
    time.sleep(1)
    
    # 정합 실행
    best_transform, best_fitness = fast_registration(source_down, target_down, vis)
    
    # 결과 출력
    print(f"\nFinal fitness: {best_fitness}")
    print("\nTransformation matrix:")
    print(best_transform)
    
    print("\nRegistration completed. Press 'q' to exit...")
    
    while True:
        if not vis.poll_events():
            break
        vis.update_renderer()
        time.sleep(0.1)
    # 시각화 창 닫기
    vis.destroy_window()

1. Aligning source to occlusal plane...
2. Running initial ICP...
3. Intermediate refinement...
4. Final refinement...

Final fitness: 0.06629448709002093

Transformation matrix:
[[ 9.95914316e-01 -6.39842460e-02  6.37235482e-02 -3.76202584e+00]
 [ 6.37159801e-02  9.97948594e-01  6.23523865e-03 -1.81522034e-01]
 [-6.39917824e-02 -2.14955511e-03  9.97948110e-01 -1.13020223e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Registration completed. Press 'q' to exit...


## 너무 심심해서 애니메이션좀 넣자

In [1]:
import open3d as o3d
import numpy as np
import copy
import time

def draw_registration_result_live(vis, source, target, transformation=None, title="Registration"):
    """실시간으로 정합 과정을 시각화합니다."""
    source_temp = copy.deepcopy(source)
    source_temp.paint_uniform_color([1, 0, 0])  # 빨간색 (소스)
    target.paint_uniform_color([0, 1, 0])  # 초록색 (타겟)
    
    if transformation is not None:
        source_temp.transform(transformation)
    
    # 기존 지오메트리 제거 및 새로운 지오메트리 추가
    vis.clear_geometries()
    vis.add_geometry(source_temp)
    vis.add_geometry(target)
    
    # 좌표축 추가
    points = np.asarray(target.points)
    x_min, y_min, z_min = np.min(points, axis=0)
    x_max, y_max, z_max = np.max(points, axis=0)
    model_size = max(x_max - x_min, y_max - y_min, z_max - z_min)
    
    coord = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=model_size * 0.2,
        origin=[x_min - model_size * 0.3, y_min - model_size * 0.3, z_min - model_size * 0.3]
    )
    vis.add_geometry(coord)
    
    # 그리드 추가
    grid_size = model_size * 2
    grid_step = model_size * 0.2
    grid_lines = []
    
    for i in range(-int(grid_size/grid_step), int(grid_size/grid_step) + 1):
        # X 방향 선
        start = [i * grid_step, -grid_size, 0]
        end = [i * grid_step, grid_size, 0]
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([start, end])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        grid_lines.append(line)
        
        # Y 방향 선
        start = [-grid_size, i * grid_step, 0]
        end = [grid_size, i * grid_step, 0]
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([start, end])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        grid_lines.append(line)
    
    for line in grid_lines:
        line.colors = o3d.utility.Vector3dVector([[0.2, 0.2, 0.2]])
        vis.add_geometry(line)
    
    # 렌더링 옵션 설정
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    opt.point_size = 4.5
    opt.line_width = 0.5
    
    # 카메라 설정
    ctr = vis.get_view_control()
    front = [0.0, 4, 3]
    up = [0, 0, 1]    
    ctr.set_front(front)
    ctr.set_up(up)
    ctr.set_lookat([0, 0, 3])
    ctr.set_zoom(0.4)

    # 업데이트
    vis.poll_events()
    vis.update_geometry(source_temp)
    vis.update_geometry(target)
    vis.update_geometry(coord)
    for line in grid_lines:
        vis.update_geometry(line)
    vis.update_renderer()

def align_to_occlusal_plane(pcd):
    """교합면을 기준으로 정렬합니다."""
    points = np.asarray(pcd.points)
    y_max = np.max(points[:, 1])
    pcd.translate([0, -y_max, 0])  # Y 최대값을 0으로 이동
    return pcd

def get_principal_axes(pcd):
    """포인트 클라우드의 주축과 중심점을 계산합니다."""
    points = np.asarray(pcd.points)
    mean = np.mean(points, axis=0)
    centered = points - mean
    cov = np.cov(centered.T)
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    # 주축 정렬 (큰 고유값 순서대로)
    idx = eigenvalues.argsort()[::-1]
    axes = eigenvectors[:, idx]
    return mean, axes

def align_heights(source, target):
    """소스와 타겟의 Z축 최저점을 맞춰 높이를 정렬합니다."""
    source_points = np.asarray(source.points)
    target_points = np.asarray(target.points)
    
    # 각각의 최저 Z값 찾기
    source_z_min = np.min(source_points[:, 2])
    target_z_min = np.min(target_points[:, 2])
    
    # Z축 방향으로 이동하여 높이 맞추기
    translation = np.array([0, 0, target_z_min - source_z_min])
    source.translate(translation)
    
    return source

def fast_registration(source, target, vis=None):
    """주축 정렬을 사용한 빠른 정합을 수행하고 과정을 시각화합니다."""
    
    # 1. 교합면 정렬 (소스에만 적용)
    print("\n[1/5] 교합면 정렬 시작...")
    source = align_to_occlusal_plane(source)
    
    if vis is not None:
        print("  - 교합면 정렬 후 시각화 중...")
        draw_registration_result_live(vis, source, target, None, "After Occlusal Alignment")
        time.sleep(0.2)
    
    # 2. Z축 높이 맞추기
    print("\n[2/5] Z축 높이 정렬 시작...")
    source = align_heights(source, target)
    
    if vis is not None:
        print("  - 높이 정렬 후 시각화 중...")
        draw_registration_result_live(vis, source, target, None, "After Height Alignment")
        time.sleep(0.2)
    
    # 3. 초기 ICP 실행
    print("\n[3/5] 초기 ICP 시작...")
    current_transform = np.eye(4)
    last_fitness = 0
    for iteration in range(50):
        result = o3d.pipelines.registration.registration_icp(
            source, target,
            2.0,
            current_transform,
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            o3d.pipelines.registration.ICPConvergenceCriteria(
                relative_fitness=1e-6,
                relative_rmse=1e-6,
                max_iteration=1
            )
        )
        
        if iteration % 100 == 0:  # 100번 반복마다 진행상황 출력
            print(f"  - 초기 ICP 반복 {iteration}: fitness = {result.fitness:.6f}")
        
        if np.allclose(result.transformation, current_transform, atol=1e-6):
            print(f"  - 초기 ICP 수렴 (반복 {iteration})")
            break
            
        current_transform = result.transformation
        
        if vis is not None:
            draw_registration_result_live(vis, source, target, current_transform, f"Initial ICP - Iteration {iteration}")
            time.sleep(0.1)
    
    # 4. 중간 미세 조정
    print("\n[4/5] 중간 미세 조정 시작...")
    for iteration in range(50):
        result = o3d.pipelines.registration.registration_icp(
            source, target,
            0.5,
            current_transform,
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            o3d.pipelines.registration.ICPConvergenceCriteria(
                relative_fitness=1e-7,
                relative_rmse=1e-7,
                max_iteration=1
            )
        )
        
        if iteration % 100 == 0:  # 100번 반복마다 진행상황 출력
            print(f"  - 중간 미세 조정 반복 {iteration}: fitness = {result.fitness:.6f}")
        
        if np.allclose(result.transformation, current_transform, atol=1e-7):
            print(f"  - 중간 미세 조정 수렴 (반복 {iteration})")
            break
            
        current_transform = result.transformation
        
        if vis is not None:
            draw_registration_result_live(vis, source, target, current_transform, f"Intermediate Refinement - Iteration {iteration}")
            time.sleep(0.001)
    
    # 5. 최종 미세 조정
    print("\n[5/5] 최종 미세 조정 시작...")
    for iteration in range(100):
        result = o3d.pipelines.registration.registration_icp(
            source, target,
            0.2,
            current_transform,
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            o3d.pipelines.registration.ICPConvergenceCriteria(
                relative_fitness=1e-8,
                relative_rmse=1e-8,
                max_iteration=1
            )
        )
        
        if iteration % 100 == 0:  # 100번 반복마다 진행상황 출력
            print(f"  - 최종 미세 조정 반복 {iteration}: fitness = {result.fitness:.6f}")
        
        if np.allclose(result.transformation, current_transform, atol=1e-8):
            print(f"  - 최종 미세 조정 수렴 (반복 {iteration})")
            break
            
        current_transform = result.transformation
        
        if vis is not None:
            draw_registration_result_live(vis, source, target, current_transform, f"Final Refinement - Iteration {iteration}")
            time.sleep(0.001)
    
    if vis is not None:
        print("\n최종 결과 시각화 중...")
        draw_registration_result_live(vis, source, target, current_transform, "Final Result")
        vis.run()
    
    print(f"\n정합 완료! 최종 fitness: {result.fitness:.6f}")
    return current_transform, result.fitness
# 사용 예시:
if __name__ == "__main__":
    # 시각화 창 생성
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    sourceMeshPath = "results/partial_smile_arch.stl"   
    targetMeshPath = "assets/library/smile_arch_origin.stl"

    source = o3d.io.read_triangle_mesh(sourceMeshPath)
    target = o3d.io.read_triangle_mesh(targetMeshPath)

    # 포인트 클라우드 생성 및 전처리
    source_pcd = source.sample_points_uniformly(number_of_points=20000)
    target_pcd = target.sample_points_uniformly(number_of_points=20000)
    
    # 포인트 클라우드 준비
    voxel_size = 0.5
    source_down = source_pcd.voxel_down_sample(voxel_size=voxel_size)
    target_down = target_pcd.voxel_down_sample(voxel_size=voxel_size)
    
    # 법선 벡터 계산
    source_down.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
    target_down.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
    
    # 초기 상태 시각화
    draw_registration_result_live(vis, source_down, target_down, None, "Initial State")
    time.sleep(1)
    
    # 정합 실행
    best_transform, best_fitness = fast_registration(source_down, target_down, vis)
    
    # 결과 출력
    print(f"\nFinal fitness: {best_fitness}")
    print("\nTransformation matrix:")
    print(best_transform)
    
    print("\nRegistration completed. Press 'q' to exit...")
    
    while True:
        if not vis.poll_events():
            break
        vis.update_renderer()
        time.sleep(0.1)
    # 시각화 창 닫기
    vis.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.

[1/5] 교합면 정렬 시작...
  - 교합면 정렬 후 시각화 중...

[2/5] Z축 높이 정렬 시작...
  - 높이 정렬 후 시각화 중...

[3/5] 초기 ICP 시작...
  - 초기 ICP 반복 0: fitness = 1.000000
  - 초기 ICP 수렴 (반복 4)

[4/5] 중간 미세 조정 시작...
  - 중간 미세 조정 반복 0: fitness = 0.999699
  - 중간 미세 조정 수렴 (반복 1)

[5/5] 최종 미세 조정 시작...
  - 최종 미세 조정 반복 0: fitness = 0.853784
  - 최종 미세 조정 수렴 (반복 2)

최종 결과 시각화 중...

정합 완료! 최종 fitness: 0.853784

Final fitness: 0.8537835393427796

Transformation matrix:
[[ 9.99999971e-01  3.24952609e-05  2.38881789e-04 -4.77643780e-04]
 [-3.24953117e-05  9.99999999e-01  2.08555029e-07  6.12949540e-01]
 [-2.38881782e-04 -2.16317561e-07  9.99999971e-01  5.31896020e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Registration completed. Press 'q' to exit...
